In [32]:
import nibabel as nib
import numpy as np
import os
import itk
import matplotlib.pyplot as plt
from monai.config import print_config
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    LoadImage,  # Adjusted from LoadImaged for direct image loading
    Rotate90,
    Flip,
    NormalizeIntensity,
    ThresholdIntensity,
    SpatialPad,
    Resize,
    Transpose,
)
image_shape = 301
depth = 40
val_transforms = Compose([
    LoadImage(image_only=True, ensure_channel_first=True, reader="ITKReader"),
    Rotate90(k=3),
    Flip(spatial_axis=1),
    NormalizeIntensity(subtrahend=15, divisor=85),
    ThresholdIntensity(threshold=0, above=True, cval=0.0),
    ThresholdIntensity(threshold=1, above=False, cval=0.0),
    SpatialPad(spatial_size=[-1, -1, depth], mode='constant', method="symmetric"),
    Resize(spatial_size=[image_shape, image_shape, depth]),
    # Transpose(indices=[0, 3, 1, 2])  # Adjust based on actual shape after previous transforms
])
map_type = "Guided-Grad-Cam"
layer="__features.transition3.conv"
if map_type == "guidedBackProp":
    dir_saliency_map_prog="/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/Results/BaselineDenseNet_Prognosis/Dropout0.2WeightDecay0.0001/SaliencyMapsPrognosis/MedCam/Fold0/"+map_type
else:
    dir_saliency_map_prog="/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/Results/BaselineDenseNet_Prognosis/Dropout0.2WeightDecay0.0001/SaliencyMapsPrognosis/MedCam/Fold0/"+map_type+layer



In [33]:
for saliency_map_path_prog in os.listdir(dir_saliency_map_prog):
    saliency_map_nii_prog = nib.load(os.path.join(dir_saliency_map_prog,saliency_map_path_prog))
    print("Shape saliency map prognosis:", saliency_map_nii_prog.shape)
    patientID=saliency_map_path_prog.split("_")[0]
    label_name=saliency_map_path_prog.split(".")[0].split("-")[-1]
    dir_original_file=os.path.join("/home/ubuntu/tenerife/data/ICH_nii_StrippedSkull/", label_name)
    original_file_path=os.path.join(dir_original_file, f'{patientID}_brain.nii.gz')
    original_image_nii = itk.imread(original_file_path)
    original_image = itk.GetArrayFromImage(original_image_nii)
    print("Shape original image:", original_image_nii.shape)


    # Convert NIfTI to numpy arrays
    saliency_map_prog = saliency_map_nii_prog.get_fdata()
    processed_image = val_transforms(original_file_path)
    # squeeze processed image to 3D
    processed_image = np.squeeze(processed_image)
    print("Shape processed image:", processed_image.shape)

    slice_index_range=range(saliency_map_nii_prog.shape[-1])
    for slice_index in slice_index_range:
        dir_to_save_png_saliencyMap = os.path.join("/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/Results/BaselineDenseNet_Prognosis/Dropout0.2WeightDecay0.0001/SaliencyMapsPrognosis/png_MedCam", map_type)
        dir_to_save_png_saliencyMap_patient = os.path.join(dir_to_save_png_saliencyMap, f"{patientID}-label-{label_name}")
        os.makedirs(dir_to_save_png_saliencyMap_patient, exist_ok=True)
        # normalize saliency_map
        saliency_map_normalized_prog = (saliency_map_prog - np.min(saliency_map_prog)) / (np.max(saliency_map_prog) - np.min(saliency_map_prog))
        # convert background of saliency_map to 0
        saliency_map_normalized_prog[saliency_map_normalized_prog < 0.5] = 0
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        ax.imshow(processed_image[:, :, slice_index], cmap='gray')
        ax.imshow(saliency_map_normalized_prog[:, :, slice_index], cmap='inferno', alpha=0.5)
        ax.axis('off')
        plt.savefig(os.path.join(dir_to_save_png_saliencyMap_patient, f"{patientID}_slice_{slice_index}_{map_type}-label-{label_name}.png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig)


Shape saliency map prognosis: (301, 301, 40)
Shape original image: (28, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (28, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (28, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (28, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (28, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (28, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (32, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301, 40)
Shape original image: (32, 512, 512)
Shape processed image: (301, 301, 40)
Shape saliency map prognosis: (301, 301,

### Read all csv files with results and convert them to excel!

In [37]:
import pandas as pd
import numpy as np
# amaia
df_amaia=pd.read_excel("/home/ubuntu/tenerife/data/ICH_nii_ReviseRadiologists/RespuestasAmaia.xlsx")
# drop last two rows of df_amaia
df_amaia.drop(df_amaia.tail(2).index,inplace=True)
df_amaia.rename(columns={'PRONÓSTICO (AMAIA)': 'AmaiaLabel', 'LABEL': 'GroundTruthLabel'}, inplace=True)
df_amaia

,ID,AmaiaLabel,GroundTruthLabel,CONCORDANCIA,MOTIVO
0,12.0,1.0,1.0,OK,NaN
1,20.0,0.0,1.0,DISCORDANCIA,Hematomas subdurales bilaterales. Como no hay ...
2,34.0,0.0,0.0,OK,NaN
3,51.0,0.0,0.0,OK,NaN
4,56.0,0.0,1.0,DISCORDANCIA,Hematoma de gran tamaño pero sin HIV ni signif...
5,59.0,0.0,1.0,DISCORDANCIA,Hematoma subdural derecho agudo.
6,71.0,1.0,0.0,DISCORDANCIA,Gran volumen y abierto a ventrículos (HIV). Pa...
7,90.0,0.0,0.0,OK,NaN
8,92.0,0.0,1.0,DISCORDANCIA,CON DUDAS HE PUESTO BUEN PRONÓSTICO. Es cierto...
9,93.0,1.0,0.0,DISCORDANCIA,HIV y mucho volumen. Para mi es mal pronóstico.


In [39]:
# read df_radiologists
df_radiologists=pd.read_csv('/home/ubuntu/tenerife/data/ICH_nii_ReviseRadiologists/test_Radiologists_fold0.csv')
df_radiologists

,PatientID,"Label_prognosis (0 good, 1 poor)",R1_8years,R2_10years,R3_5years
0,12,1,0,1,0
1,20,1,1,0,0
2,34,0,0,0,0
3,51,0,0,0,0
4,56,1,0,0,0
5,59,1,0,0,0
6,71,0,0,0,0
7,90,0,0,0,0
8,92,1,0,0,0
9,93,0,1,1,1


In [43]:
df_amaia['GroundTruthLabel'].equals(df_radiologists['Label_prognosis (0 good, 1 poor)'].astype(float))

True

In [57]:
clinical_data = pd.read_csv('/home/ubuntu/tenerife/data/ICH_tabular_data/CLINICAL_DATA_ICH.csv', delimiter=',')
# select only patientID in df_radiologists['PatientID']
clinical_data=clinical_data[clinical_data['PatientID'].isin(df_radiologists['PatientID'])]
# rest index in clinical_data
clinical_data.reset_index(drop=True, inplace=True)
df_amaia['GroundTruthLabel'].equals(clinical_data['Label (poor_prognosis)'].astype(float))

True

In [60]:
# read baseline results
df_baseline=pd.read_csv("/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/Results/BaselineDenseNet_Prognosis/Dropout0.2WeightDecay0.0001/test_labels_df_BaselineDenseNet_Prognosis301_40_Dropout0.2WeightDecay0.0001.csv")
# select only Fold=0
df_baseline_fold0=df_baseline[df_baseline["Fold"]==0]
df_amaia['GroundTruthLabel'].equals(df_baseline_fold0['True_labels'])

True

In [62]:
# read binary GCS, binary Age, results
df_binGCS_binAge=pd.read_csv("/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/Results/BinaryGCS_BinaryAge_Prognosis/test_labels_df_BinaryGCSBinaryAge_Prognosis_MulticlassOutput.csv")
# select only Fold=0
df_binGCS_binAge_fold0=df_binGCS_binAge[df_binGCS_binAge["Fold"]==0]
df_amaia['GroundTruthLabel'].equals(df_binGCS_binAge_fold0['True_labels'])

True

In [64]:
# read three class GCS, binary Age, results
df_3classGCS_binAge=pd.read_csv("/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/Results/ThreeClassGCS_BinaryAge_Prognosis_SameHP/test_labels_df_ThreeClassGCS_BinaryAge_Prognosis_SAMEPARAMETERS_asBinaryGCS_BinaryAge_Prognosis_No_pos_weight.csv")
# select only Fold=0
df_3classGCS_binAge_fold0=df_3classGCS_binAge[df_3classGCS_binAge["Fold"]==0]
df_amaia['GroundTruthLabel'].equals(df_3classGCS_binAge_fold0['True_labels'])

True

In [65]:
# create a df to merge all labels
df_merged = pd.DataFrame()
df_merged['PatientID'] = df_radiologists['PatientID']
df_merged['GroundTruthLabel'] = df_amaia['GroundTruthLabel']
df_merged['Age'] = clinical_data['Age']
df_merged['GCS'] = clinical_data['GCS']
df_merged['R1_8years'] = df_radiologists['R1_8years'].astype(float)
df_merged['R2_10years'] = df_radiologists['R2_10years'].astype(float)
df_merged['R3_5years'] = df_radiologists['R3_5years'].astype(float)
df_merged['AmaiaLabel'] = df_amaia['AmaiaLabel']
df_merged['Baseline_PredictedLabel'] = df_baseline_fold0['Predicted_labels_test_th0.5']
df_merged['Baseline_Probability'] = df_baseline_fold0['Probabilities_label1_test']
df_merged['BinGCS_BinAge_PredictedLabel'] = df_binGCS_binAge_fold0['Predicted_labels_test_th0.5']
df_merged['BinGCS_BinAge_Probability'] = df_binGCS_binAge_fold0['Probabilities_labels_test']
df_merged['3classGCS_BinAge_PredictedLabel'] = df_3classGCS_binAge_fold0['Predicted_labels_test_th0.5']
df_merged['3classGCS_BinAge_Probability'] = df_3classGCS_binAge_fold0['Probabilities_labels_test']
df_merged['MotivoAmaia'] = df_amaia['MOTIVO']
df_merged

,PatientID,GroundTruthLabel,Age,GCS,R1_8years,R2_10years,R3_5years,AmaiaLabel,Baseline_PredictedLabel,Baseline_Probability,BinGCS_BinAge_PredictedLabel,BinGCS_BinAge_Probability,3classGCS_BinAge_PredictedLabel,3classGCS_BinAge_Probability,MotivoAmaia
0,12,1.0,78,11,0.0,1.0,0.0,1.0,1.0,0.536089,1.0,0.639562,0.0,0.457137,NaN
1,20,1.0,85,11,1.0,0.0,0.0,0.0,1.0,0.531995,0.0,0.253471,0.0,0.409046,Hematomas subdurales bilaterales. Como no hay ...
2,34,0.0,78,15,0.0,0.0,0.0,0.0,1.0,0.543555,0.0,0.258576,0.0,0.396868,NaN
3,51,0.0,71,13,0.0,0.0,0.0,0.0,0.0,0.467638,0.0,0.251951,0.0,0.362136,NaN
4,56,1.0,85,3,0.0,0.0,0.0,0.0,1.0,0.579435,1.0,0.531665,0.0,0.468165,Hematoma de gran tamaño pero sin HIV ni signif...
5,59,1.0,78,13,0.0,0.0,0.0,0.0,1.0,0.607184,1.0,0.645855,1.0,0.558870,Hematoma subdural derecho agudo.
6,71,0.0,84,11,0.0,0.0,0.0,1.0,1.0,0.564783,0.0,0.437534,0.0,0.459550,Gran volumen y abierto a ventrículos (HIV). Pa...
7,90,0.0,77,15,0.0,0.0,0.0,0.0,1.0,0.570493,0.0,0.445617,0.0,0.473401,NaN
8,92,1.0,45,11,0.0,0.0,0.0,0.0,1.0,0.755583,1.0,0.797322,1.0,0.713516,CON DUDAS HE PUESTO BUEN PRONÓSTICO. Es cierto...
9,93,0.0,62,12,1.0,1.0,1.0,1.0,1.0,0.567235,0.0,0.440382,0.0,0.457207,HIV y mucho volumen. Para mi es mal pronóstico.


In [66]:
# save df_merged as excel
df_merged.to_excel("/home/ubuntu/tenerife/data/ZZ_ICH_PrognosisMICCAI/ExcelResultsMICCAI/ResultsModelsVsRadiologists.xlsx", index=False)